In [34]:
from skt.ye import get_spark
from skt.gcp import load_bigquery_ipython_magic, \
                    bq_to_pandas, \
                    get_bigquery_client

load_bigquery_ipython_magic()

In [43]:
%%bq max_dt
    SELECT MAX(dt) as max_dt
    FROM tworld.twd_dst_product_group

BigQuery execution took 2 seconds.


In [2]:
%%bq
SELECT *
FROM tworld.twd_dst_product_group
WHERE dt = (SELECT MAX(dt) FROM tworld.twd_dst_product_group)

BigQuery execution took 1 seconds.


,product_grp_id,product_grp_nm,dt
0,000003412,노키아 바나나폰,2020-07-22
1,000001572,BlackBerry PRIV,2020-07-22
2,000002832,Re 아이폰 8,2020-07-22
3,000002833,Re 아이폰 8 플러스,2020-07-22
4,000002834,Re 아이폰 X,2020-07-22
...,...,...,...
359,000002072,갤럭시 와이드2,2020-07-22
360,000003332,애플워치4 44mm,2020-07-22
361,000004234,갤럭시 S20 플러스 5G,2020-07-22
362,000002412,엘지 스마트 폴더,2020-07-22


In [3]:
import time

str(int(time.time()))

'1595587052'

In [4]:
BigQuery to Pandas

SyntaxError: invalid syntax (<ipython-input-4-f83ff3b84f9a>, line 1)

In [5]:
pd_df = bq_to_pandas("""
    SELECT *
    FROM tworld.twd_dst_product_group
    WHERE dt = (SELECT MAX(dt) FROM tworld.twd_dst_product_group)
""")

pd_df

Downloading: 100%|██████████| 364/364 [00:02<00:00, 125.27rows/s]


,product_grp_id,product_grp_nm,dt
0,000003412,노키아 바나나폰,2020-07-22
1,000001572,BlackBerry PRIV,2020-07-22
2,000002832,Re 아이폰 8,2020-07-22
3,000002833,Re 아이폰 8 플러스,2020-07-22
4,000002834,Re 아이폰 X,2020-07-22
...,...,...,...
359,000002072,갤럭시 와이드2,2020-07-22
360,000003332,애플워치4 44mm,2020-07-22
361,000004234,갤럭시 S20 플러스 5G,2020-07-22
362,000002412,엘지 스마트 폴더,2020-07-22


Pandas to BigQuery

In [7]:
dest_table = f"temp_1d.twd_dst_product_group_{str(int(time.time()))}"
pd_df.to_gbq(dest_table)

print(dest_table)


get_bigquery_client().query(f"""
    SELECT *
    FROM {dest_table}
""").result()

1it [00:04,  4.71s/it]


temp_1d.twd_dst_product_group_1595587059


In [8]:
query_params =  {"dest_table": dest_table}

In [10]:
%%bq --params $query_params
    SELECT *
    FROM {dest_table}

BigQuery execution took 1 seconds.


,product_grp_id,product_grp_nm,dt
0,000003412,노키아 바나나폰,2020-07-22 00:00:00+00:00
1,000001572,BlackBerry PRIV,2020-07-22 00:00:00+00:00
2,000002832,Re 아이폰 8,2020-07-22 00:00:00+00:00
3,000002833,Re 아이폰 8 플러스,2020-07-22 00:00:00+00:00
4,000002834,Re 아이폰 X,2020-07-22 00:00:00+00:00
...,...,...,...
359,000002072,갤럭시 와이드2,2020-07-22 00:00:00+00:00
360,000003332,애플워치4 44mm,2020-07-22 00:00:00+00:00
361,000004234,갤럭시 S20 플러스 5G,2020-07-22 00:00:00+00:00
362,000002412,엘지 스마트 폴더,2020-07-22 00:00:00+00:00


In [ ]:
from skt.gcp import bq_to_spark_df

In [11]:
from skt.gcp import bq_to_df

In [14]:
spark_df = bq_to_df(f"""
    SELECT *
    FROM {dest_table}
""")

spark_df.head(5)

[Row(product_grp_id='000000292', product_grp_nm='XPERIA™ Z3', dt=datetime.datetime(2020, 7, 22, 9, 0)),
 Row(product_grp_id='000000212', product_grp_nm='G3 A', dt=datetime.datetime(2020, 7, 22, 9, 0)),
 Row(product_grp_id='000000992', product_grp_nm='Gear S2 band Classic', dt=datetime.datetime(2020, 7, 22, 9, 0)),
 Row(product_grp_id='000001412', product_grp_nm='SKY IM-100', dt=datetime.datetime(2020, 7, 22, 9, 0)),
 Row(product_grp_id='000004173', product_grp_nm='RE 아이폰 11 프로', dt=datetime.datetime(2020, 7, 22, 9, 0))]

### Spark to BigQuery
#### Spark Dataframe을 BigQuery Partitioned Table에 저장하는 예제입니다.

1. 저장할 테이블을 생성합니다. Date 타입 파티션된 테이블입니다.

In [53]:
from skt.gcp import df_to_bq_table
from pyspark.sql.types import DateType

dest_dataset = "temp_1d"
partitioned_dest_table = f"twd_dst_product_group_{str(int(time.time()))}"

get_bigquery_client().query(f"""
    CREATE OR REPLACE TABLE {dest_dataset}.{partitioned_dest_table}
    (
        product_grp_id STRING,
        product_grp_nm STRING,
        dt DATE
    )
    PARTITION BY dt
""").result()

2. Spark Dataframe을 BigQuery 테이블에 저장합니다. 타입에 주의하며 필요 시 변환해주는 작업이 필요할 수 있습니다.

In [61]:
changed_df = spark_df.select("product_grp_id", "product_grp_nm", spark_df.dt.cast("date"))
partition_dt = changed_df.head(1)[0].dt.strftime("%Y%m%d")
df_to_bq_table(df=changed_df,
               dataset=dest_dataset,
               table_name=partitioned_dest_table,
               partition=partition_dt,
               mode="overwrite")

3. BigQuery에 저장되었는지 확인합니다.

In [62]:
query_params = {"dataset":dest_dataset, "table_name":partitioned_dest_table}

In [63]:
%%bq --params $query_params
    SELECT *
    FROM {dataset}.{table_name}

BigQuery execution took 1 seconds.


,product_grp_id,product_grp_nm,dt
0,000000292,XPERIA™ Z3,2020-07-22
1,000000212,G3 A,2020-07-22
2,000000992,Gear S2 band Classic,2020-07-22
3,000001412,SKY IM-100,2020-07-22
4,000004173,RE 아이폰 11 프로,2020-07-22
...,...,...,...
359,000004272,엘지 Q51,2020-07-22
360,000002332,LG Q6+,2020-07-22
361,000003152,엘지 Q8,2020-07-22
362,000003452,갤럭시 A9,2020-07-22
